# AnalogBase Overview

## What you will learn
* What AnalogBase is
* Common layout considerations when using AnalogBase
* Writing a loayout generator for a source follower

## AnalogBase Overview
<img src="bootcamp_pics/3_analogbase/analogbase_1.PNG" />
* Draw rows of NMOS followed by rows of PMOS
    * Can have only NMOS or only PMOS
    * Index 0 is bottom-most row
* Draw substrates on ends for body contact

## AnalogMosConn Overview
<img src="bootcamp_pics/3_analogbase/analogbase_2.PNG" />
* AnalogBase "drops" AnalogMosConn on top of desired transistors to make connections to a vertical metal layer
* Gate wires location determined by source/drain direction
    * If source goes down, gate must be on drain side

## Transistor Naming Convention
<img src="bootcamp_pics/3_analogbase/analogbase_3.PNG" />
* By convention, left-most source/drain junction is called "source"
    * Right-most source/drain junction will be "source" only if you have even number of fingers

## Connecting to High Level Tracks
<img src="bootcamp_pics/3_analogbase/analogbase_4.PNG" />
* By default gate connections from bottom of each row
    * Flip the row to connect from top
* Issues:
    * How to figure out which tracks I can connect to?
    * How to allocate more horizontal tracks?
<img src="bootcamp_pics/3_analogbase/analogbase_5.PNG" />
* Solution: relative indexing
* track 0 is the track closest to gate-side boundary
* User specify number of tracks needed for gate/drain/source connections on each row

## CS Amplifier Layout Example
<img src="bootcamp_pics/3_analogbase/analogbase_6.PNG" />
* ALWAYS DRAW FLOORPLAN FIRST
* Put dummies on both sides
* Output between row, and inputs above/below
* Gate wires aligned with output wires

## Class Definition
```python
class AmpCS(AnalogBase):
    """A common source amplifier"""
    def __init__(self, temp_db, lib_name, params, used_names **kwargs):
        super(AmpCS, self).__init__(temp_db, lib_name, params, used_names, **kwargs)
        self._sch_params = None
        
        @property
        def sch_params(self):
            return self._sch_params
```
* Subclass AnalogBase to inherit useful functions
* Simple Python class constructor
* Initialize read-only sch_params property
* Layout code will compute sch_params to make LVS clean

## Parameter Specifications
```python
@classmethod
def get_params_info(cls):
        """Returns a dictionary containing parameter descriptions.
        Override this method to return a dictionary from parameter names to descriptions.
        Returns
        -------
        param_info : dict[str, str]
            dictionary from parameter name to description.
        """
        return dict(
            lch='channel length, in meters.',
            w_dict='width dictionary.',
            intent_dict='intent dictionary.',
            fg_dict='number of fingers dictionary.',
            ndum='number of dummies on each side.',
            ptap_w='NMOS substrate width, in meters/number of fins.',
            ntap_w='PMOS substrate width, in meters/number of fins.',
            show_pins='True to draw pin geometries.',
        )
```
* Must implement get_params_info() method
* List all layout parametercs

## How many fingers in a row?
```python
 # compute total number of fingers in each row
fg_half_pmos = fg_load // 2
fg_half_nmos = fg_amp // 2
fg_half = max(fg_half_pmos, fg_half_nmos)
fg_tot = (fg_half + ndum) * 2
```
<img src="bootcamp_pics/3_analogbase/analogbase_7.PNG" />
* Total number of fingers depends on maximum of amp/load fingers

## Drawing Transistor Rows
```python
# specify width/threshold of each row
nw_list = [w_dict['amp']]
pw_list = [w_dict['load']]
nth_list = [intent_dict['amp']]
pth_list = [intent_dict['load']]

# specify number of horizontal tracks for each row
ng_tracks = [1]  # input track
nds_tracks = [1]  # one track for space
pds_tracks = [1]  # output track
pg_tracks = [1]  # bias track

# specify row orientations
n_orient = ['R0'] # gate connection on bottom
p_orient = ['MX'] # gate connection on top

self.draw_base(lch, fg_tot, ptap_w, ntap_w, nw_list,
               nth_list, pw_list, pth_list,
               ng_tracks=ng_tracks, nds_tracks=nds_tracks,
               pg_tracks=pg_tracks, pds_tracks=pds_tracks,
               n_orientations=n_orient, p_orientations=p_orient,
               )
```
* Specify row parameters, then draw rows

## Is output on source or drain?
```python
# figure out if output connects to drain or source of nmos
if (fg_amp - fg_load) % 4 == 0:
    aout, aoutb, nsdir, nddir = 'd', 's', 0, 2
else:
    aout, aoutb, nsdir, nddir = 's', 'd', 2, 0
```
<img src="bootcamp_pics/3_analogbase/analogbase_8.PNG" />
* We always connect output to drain of load
* Depending on (fg_load-fg_amp) % 4, output is connected to drain or source of amp

## Drawing Transistor Connections
```python
# create transistor connections
load_col = ndum + fg_half - fg_half_pmos
amp_col = ndum + fg_half - fg_half_nmos
amp_ports = self.draw_mos_conn('nch', 0, amp_col, fg_amp, nsdir, nddir)
load_ports = self.draw_mos_conn('pch', 0, load_col, fg_load, 2, 0)
# amp_ports/load_ports are dictionaries of WireArrays representing
# transistor ports.
print(amp_ports)
print(amp_ports['g'])
```
* Create transistor connections
* draw_mos_conn() connects transistors, then returns dictionary from transistor port names to WireArrays
* For sdir/ddir, 0 is down, 2 is up

## Connecting Wires
```python
# create TrackID from relative track index
vin_tid = self.make_track_id('nch', 0, 'g', 0)
vout_tid = self.make_track_id('pch', 0, 'ds', 0)
vbias_tid = self.make_track_id('pch', 0, 'g', 0)
# can also convert from relative to absolute track index
print(self.get_track_index('nch', 0, 'g', 0))

vin_warr = self.connect_to_tracks(amp_ports['g'], vin_tid)
vout_warr = self.connect_to_tracks([amp_ports[aout], load_ports['d']], vout_tid)
vbias_warr = self.connect_to_tracks(load_ports['g'], vbias_tid)
self.connect_to_substrate('ptap', amp_ports[aoutb])
self.connect_to_substrate('ntap', load_ports['s'])
```
* make_track_id() and get_track_index() converts from relative to absolute index
* connect_to_substrate() connects transistor ports to substrate on vertical metal layer

## Dummies and Pins
```python
vss_warrs, vdd_warrs = self.fill_dummy()

self.add_pin('VSS', vss_warrs, show=show_pins)
self.add_pin('VDD', vdd_warrs, show=show_pins)
self.add_pin('vin', vin_warr, show=show_pins)
self.add_pin('vout', vout_warr, show=show_pins)
self.add_pin('vbias', vbias_warr, show=show_pins)
```
* fill_dummy() connects all unused transistors to substrates, then connect substrates to horizontal metal layer
* show_pins layout parameter is useful for hiding pins in subcells

## Schematic Parameters
```python
 # compute schematic parameters
sch_fg_dict = fg_dict.copy()
sch_fg_dict['dump'] = fg_tot - fg_load
if aout == 'd':
    sch_fg_dict['dumn_list'] = [fg_tot - fg_amp]
else:
    sch_fg_dict['dumn_list'] = [fg_tot - fg_amp - 2, 2]
self._sch_params = dict(
    lch=lch,
    w_dict=w_dict,
    intent_dict=intent_dict,
    fg_dict=sch_fg_dict,
)
```
* Compute schematic parameters
* Note that dummy schematic changes if nmos source is connected to output

## SF Amplifier Exercise
* Now try to code a source-follower amplifier by filling in missing code in class AmpSF below
* Use the below floorplan
* If you're stuck you can look at the AmpSFSoln class below for a complete solution
<img src="bootcamp_pics/3_analogbase/analogbase_9.PNG" />
* Two rows of NMOS
* Gate on top for second row
* 1 track spacing between vin and VDD

In [1]:
import sys
sys.path.append('BAG_XBase_demo/demo_scripts')
sys.path.append('BAG_XBase_demo/xbase_demo/demo_layout')
import bootcamp_demo as d
#import core
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
# noinspection PyUnresolvedReferences,PyCompatibility
from builtins import *

from bag.layout.routing import TrackID
from bag.layout.template import TemplateBase

from abs_templates_ec.analog_core import AnalogBase

In [10]:
class AmpSF_new(AnalogBase):
    """A template of a single transistor with dummies.
    This class is mainly used for transistor characterization or
    design exploration with config views.
    Parameters
    ----------
    temp_db : :class:`bag.layout.template.TemplateDB`
            the template database.
    lib_name : str
        the layout library name.
    params : dict[str, any]
        the parameter values.
    used_names : set[str]
        a set of already used cell names.
    kwargs : dict[str, any]
        dictionary of optional parameters.  See documentation of
        :class:`bag.layout.template.TemplateBase` for details.
    """

    def __init__(self, temp_db, lib_name, params, used_names, **kwargs):
        super(AmpSF_new, self).__init__(temp_db, lib_name, params, used_names, **kwargs)
        self._sch_params = None

    @property
    def sch_params(self):
        return self._sch_params

    @classmethod
    def get_params_info(cls):
        """Returns a dictionary containing parameter descriptions.
        Override this method to return a dictionary from parameter names to descriptions.
        Returns
        -------
        param_info : dict[str, str]
            dictionary from parameter name to description.
        """
        return dict(
            lch='channel length, in meters.',
            w_dict='width dictionary.',
            intent_dict='intent dictionary.',
            fg_dict='number of fingers dictionary.',
            ndum='number of dummies on each side.',
            ptap_w='NMOS substrate width, in meters/number of fins.',
            ntap_w='PMOS substrate width, in meters/number of fins.',
            show_pins='True to draw pin geometries.',
        )

    def draw_layout(self):
        """Draw the layout of a transistor for characterization.
        """

        lch = self.params['lch']
        w_dict = self.params['w_dict']
        intent_dict = self.params['intent_dict']
        fg_dict = self.params['fg_dict']
        ndum = self.params['ndum']
        ptap_w = self.params['ptap_w']
        ntap_w = self.params['ntap_w']
        show_pins = self.params['show_pins']

        fg_amp = fg_dict['amp']
        fg_bias = fg_dict['bias']

        if fg_bias % 2 != 0 or fg_amp % 2 != 0:
            raise ValueError('fg_bias=%d and fg_amp=%d must all be even.' % (fg_bias, fg_amp))

        fg_half_bias = fg_bias // 2
        fg_half_amp = fg_amp // 2
        fg_half = max(fg_half_bias, fg_half_amp)
        fg_tot = (fg_half + ndum) * 2

        nw_list = [w_dict['bias'], w_dict['amp']]
        nth_list = [intent_dict['bias'], intent_dict['amp']]
        ng_tracks = [1, 3]
        nds_tracks = [1, 1]

        n_orient = ['R0', 'MX']

        self.draw_base(lch, fg_tot, ptap_w, ntap_w, nw_list,
                       nth_list, [], [],
                       ng_tracks=ng_tracks, nds_tracks=nds_tracks,
                       pg_tracks=[], pds_tracks=[],
                       n_orientations=n_orient,
                       )

        if (fg_amp - fg_bias) % 4 == 0:
            aout, aoutb, nsdir, nddir = 'd', 's', 2, 0
        else:
            aout, aoutb, nsdir, nddir = 's', 'd', 0, 2

        # TODO: compute bias_col and amp_col
        bias_col = amp_col = 0

        amp_ports = self.draw_mos_conn('nch', 1, amp_col, fg_amp, nsdir, nddir)
        bias_ports = self.draw_mos_conn('nch', 0, bias_col, fg_bias, 0, 2)

        # TODO: get TrackIDs for horizontal tracks
        # The following are related code copied and pasted from AmpCS
        # for reference
        # vin_tid = self.make_track_id('nch', 0, 'g', 0)
        # vout_tid = self.make_track_id('pch', 0, 'ds', 0)
        # vbias_tid = self.make_track_id('pch', 0, 'g', 0)
        vdd_tid = vin_tid = vout_tid = vbias_tid = None

        if vdd_tid is None:
            return

        # uncomment to visualize track location
        # hm_layer = self.mos_conn_layer + 1
        # xl = self.bound_box.left_unit
        # xr = self.bound_box.right_unit
        # self.add_wires(hm_layer, vdd_tid.base_index, xl, xr, unit_mode=True)
        # self.add_wires(hm_layer, vin_tid.base_index, xl, xr, unit_mode=True)
        # self.add_wires(hm_layer, vout_tid.base_index, xl, xr, unit_mode=True)
        # self.add_wires(hm_layer, vbias_tid.base_index, xl, xr, unit_mode=True)
        
        # TODO: connect transistors to horizontal tracks
        # The following are related code copied and pasted from AmpCS
        # for reference
        # vin_warr = self.connect_to_tracks(amp_ports['g'], vin_tid)
        # vout_warr = self.connect_to_tracks([amp_ports[aout], load_ports['d']], vout_tid)
        # vbias_warr = self.connect_to_tracks(load_ports['g'], vbias_tid)
        vin_warr = vout_warr = vbias_warr = vdd_warr = None

        if vin_warr is None:
            return

        self.connect_to_substrate('ptap', bias_ports['s'])

        vss_warrs, _ = self.fill_dummy()

        self.add_pin('VSS', vss_warrs, show=show_pins)
        # TODO: add pins

        sch_fg_dict = fg_dict.copy()
        sch_fg_dict['dum_list'] = [fg_tot - fg_bias, fg_tot - fg_amp - 2, 2]

        self._sch_params = dict(
            lch=lch,
            w_dict=w_dict,
            intent_dict=intent_dict,
            fg_dict=sch_fg_dict,
        )
        

* Now run this cell to see the results

In [9]:
spec_fname = 'demo_specs/demo.yaml'

# load specifications from file
top_specs = d.read_yaml(spec_fname)

# create BagProject object
local_dict = locals()
if 'bprj' in local_dict:
    print('using existing BagProject')
    bprj = local_dict['bprj']
else:
    print('creating BagProject')
    bprj = d.BagProject()
    
d.gen_layout(bprj, top_specs, 'amp_sf', AmpSF_new)


using existing BagProject
computing layout
ext_w0 = 1, ext_wend=9, tot_ntr=20
ext_w0 = 2, ext_wend=11, tot_ntr=21
abort
creating layout
layout done


{'fg_dict': {'amp': 16, 'bias': 12, 'dum_list': [12, 6, 2]},
 'intent_dict': {'amp': 'lvt', 'bias': 'lvt'},
 'lch': 1.8e-08,
 'w_dict': {'amp': 4, 'bias': 4}}

## Solution:
```python
class AmpSFSoln(AnalogBase):
    """A template of a single transistor with dummies.
    This class is mainly used for transistor characterization or
    design exploration with config views.
    Parameters
    ----------
    temp_db : :class:`bag.layout.template.TemplateDB`
            the template database.
    lib_name : str
        the layout library name.
    params : dict[str, any]
        the parameter values.
    used_names : set[str]
        a set of already used cell names.
    kwargs : dict[str, any]
        dictionary of optional parameters.  See documentation of
        :class:`bag.layout.template.TemplateBase` for details.
    """

    def __init__(self, temp_db, lib_name, params, used_names, **kwargs):
        super(AmpSFSoln, self).__init__(temp_db, lib_name, params, used_names, **kwargs)
        self._sch_params = None

    @property
    def sch_params(self):
        return self._sch_params

    @classmethod
    def get_params_info(cls):
        """Returns a dictionary containing parameter descriptions.
        Override this method to return a dictionary from parameter names to descriptions.
        Returns
        -------
        param_info : dict[str, str]
            dictionary from parameter name to description.
        """
        return dict(
            lch='channel length, in meters.',
            w_dict='width dictionary.',
            intent_dict='intent dictionary.',
            fg_dict='number of fingers dictionary.',
            ndum='number of dummies on each side.',
            ptap_w='NMOS substrate width, in meters/number of fins.',
            ntap_w='PMOS substrate width, in meters/number of fins.',
            show_pins='True to draw pin geometries.',
        )

    def draw_layout(self):
        """Draw the layout of a transistor for characterization.
        """

        lch = self.params['lch']
        w_dict = self.params['w_dict']
        intent_dict = self.params['intent_dict']
        fg_dict = self.params['fg_dict']
        ndum = self.params['ndum']
        ptap_w = self.params['ptap_w']
        ntap_w = self.params['ntap_w']
        show_pins = self.params['show_pins']

        fg_amp = fg_dict['amp']
        fg_bias = fg_dict['bias']

        if fg_bias % 2 != 0 or fg_amp % 2 != 0:
            raise ValueError('fg_bias=%d and fg_amp=%d must all be even.' % (fg_bias, fg_amp))

        fg_half_bias = fg_bias // 2
        fg_half_amp = fg_amp // 2
        fg_half = max(fg_half_bias, fg_half_amp)
        fg_tot = (fg_half + ndum) * 2

        nw_list = [w_dict['bias'], w_dict['amp']]
        nth_list = [intent_dict['bias'], intent_dict['amp']]

        ng_tracks = [1, 3]
        nds_tracks = [1, 1]

        n_orient = ['R0', 'MX']

        self.draw_base(lch, fg_tot, ptap_w, ntap_w, nw_list,
                       nth_list, [], [],
                       ng_tracks=ng_tracks, nds_tracks=nds_tracks,
                       pg_tracks=[], pds_tracks=[],
                       n_orientations=n_orient,
                       )

        if (fg_amp - fg_bias) % 4 == 0:
            aout, aoutb, nsdir, nddir = 'd', 's', 2, 0
        else:
            aout, aoutb, nsdir, nddir = 's', 'd', 0, 2

        bias_col = ndum + fg_half - fg_half_bias
        amp_col = ndum + fg_half - fg_half_amp
        amp_ports = self.draw_mos_conn('nch', 1, amp_col, fg_amp, nsdir, nddir)
        bias_ports = self.draw_mos_conn('nch', 0, bias_col, fg_bias, 0, 2)

        vdd_tid = self.make_track_id('nch', 1, 'g', 0)
        vin_tid = self.make_track_id('nch', 1, 'g', 2)
        vout_tid = self.make_track_id('nch', 0, 'ds', 0)
        vbias_tid = self.make_track_id('nch', 0, 'g', 0)

        vin_warr = self.connect_to_tracks(amp_ports['g'], vin_tid)
        vout_warr = self.connect_to_tracks([amp_ports[aout], bias_ports['d']], vout_tid)
        vbias_warr = self.connect_to_tracks(bias_ports['g'], vbias_tid)
        vdd_warr = self.connect_to_tracks(amp_ports[aoutb], vdd_tid)
        self.connect_to_substrate('ptap', bias_ports['s'])

        vss_warrs, _ = self.fill_dummy()

        self.add_pin('VSS', vss_warrs, show=show_pins)
        self.add_pin('VDD', vdd_warr, show=show_pins)
        self.add_pin('vin', vin_warr, show=show_pins)
        self.add_pin('vout', vout_warr, show=show_pins)
        self.add_pin('vbias', vbias_warr, show=show_pins)

        sch_fg_dict = fg_dict.copy()
        sch_fg_dict['dum_list'] = [fg_tot - fg_bias, fg_tot - fg_amp - 2, 2]

        self._sch_params = dict(
            lch=lch,
            w_dict=w_dict,
            intent_dict=intent_dict,
            fg_dict=sch_fg_dict,
        )
```